In [ ]:
# from kfp import dsl
# from kfp.v2.dsl import (Artifact, Dataset, Input, InputPath, Model, Output, OutputPath, ClassificationMetrics,
#                         Metrics, component)
import os
import re
import google
from google.oauth2 import credentials
from google.oauth2 import service_account
from google.oauth2.service_account import Credentials
from datetime import date
from datetime import timedelta
from dateutil.relativedelta import relativedelta

SERVICE_TYPE = 'tos_cross_sell'
DATASET_ID = 'tos_cross_sell'
PROJECT_ID = 'divg-josh-pr-d1cc3a' #mapping['PROJECT_ID']
RESOURCE_BUCKET = 'divg-josh-pr-d1cc3a-default' #mapping['resources_bucket']
FILE_BUCKET = 'divg-josh-pr-d1cc3a-default' #mapping['gcs_csv_bucket']
REGION = 'northamerica-northeast1' #mapping['REGION']
MODEL_ID = '5060'
FOLDER_NAME = 'xgb_tos_cross_sell_train_deploy'.format(MODEL_ID)
QUERIES_PATH = 'vertex_pipelines/' + FOLDER_NAME + '/queries/'

scoringDate = date(2022, 9, 1)  # date.today() - relativedelta(days=2)- relativedelta(months=30)
valScoringDate = date(2022, 12, 1)  # scoringDate - relativedelta(days=2)

# training views
CONSL_VIEW_NAME = '{}_pipeline_consl_data_training_bi_layer'.format(SERVICE_TYPE)  # done
FFH_BILLING_VIEW_NAME = '{}_pipeline_ffh_billing_data_training_bi_layer'.format(SERVICE_TYPE)  # done
HS_USAGE_VIEW_NAME = '{}_pipeline_hs_usage_data_training_bi_layer'.format(SERVICE_TYPE)  # done
DEMO_INCOME_VIEW_NAME = '{}_pipeline_demo_income_data_training_bi_layer'.format(SERVICE_TYPE)  # done
PROMO_EXPIRY_VIEW_NAME = '{}_pipeline_promo_expiry_data_training_bi_layer'.format(SERVICE_TYPE)  # done
TROUBLE_TICKETS_VIEW_NAME = '{}_pipeline_trouble_tickets_data_training_bi_layer'.format(SERVICE_TYPE)  # done
GPON_COPPER_VIEW_NAME = '{}_pipeline_gpon_copper_data_training_bi_layer'.format(SERVICE_TYPE)  # done
CALL_DATA_VIEW_NAME = '{}_pipeline_call_data_training_bi_layer'.format(SERVICE_TYPE)  # done
HSIA_DROPS_VIEW_NAME = '{}_pipeline_hsia_drops_training_bi_layer'.format(SERVICE_TYPE)
CLCKSTRM_TELUS_VIEW_NAME = '{}_pipeline_clckstrm_telus_training_bi_layer'.format(SERVICE_TYPE)
TOS_ACTIVE_BANS_VIEW_NAME = '{}_pipeline_tos_active_bans_training_bi_layer'.format(SERVICE_TYPE) 

# validation views
CONSL_VIEW_VALIDATION_NAME = '{}_pipeline_consl_data_validation_bi_layer'.format(SERVICE_TYPE)
FFH_BILLING_VIEW_VALIDATION_NAME = '{}_pipeline_ffh_billing_data_validation_bi_layer'.format(SERVICE_TYPE)
HS_USAGE_VIEW_VALIDATION_NAME = '{}_pipeline_hs_usage_data_validation_bi_layer'.format(SERVICE_TYPE)
DEMO_INCOME_VIEW_VALIDATION_NAME = '{}_pipeline_demo_income_data_validation_bi_layer'.format(SERVICE_TYPE)
PROMO_EXPIRY_VIEW_VALIDATION_NAME = '{}_pipeline_promo_expiry_data_validation_bi_layer'.format(SERVICE_TYPE)
TROUBLE_TICKETS_VIEW_VALIDATION_NAME = '{}_pipeline_trouble_tickets_data_validation_bi_layer'.format(SERVICE_TYPE)
GPON_COPPER_VIEW_VALIDATION_NAME = '{}_pipeline_gpon_copper_data_validation_bi_layer'.format(SERVICE_TYPE)
CALL_DATA_VIEW_VALIDATION_NAME = '{}_pipeline_call_data_validation_bi_layer'.format(SERVICE_TYPE)
HSIA_DROPS_VIEW_VALIDATION_NAME = '{}_pipeline_hsia_drops_validation_bi_layer'.format(SERVICE_TYPE)
CLCKSTRM_TELUS_VIEW_VALIDATION_NAME = '{}_pipeline_clckstrm_telus_validation_bi_layer'.format(SERVICE_TYPE)
TOS_ACTIVE_BANS_VALIDATION_VIEW_NAME = '{}_pipeline_tos_active_bans_validation_bi_layer'.format(SERVICE_TYPE) 

# training dates
SCORE_DATE = scoringDate.strftime('%Y%m%d')  # date.today().strftime('%Y%m%d')
SCORE_DATE_DASH = scoringDate.strftime('%Y-%m-%d')
SCORE_DATE_MINUS_6_MOS_DASH = ((scoringDate - relativedelta(months=6)).replace(day=1)).strftime('%Y-%m-%d')
SCORE_DATE_LAST_MONTH_START_DASH = (scoringDate.replace(day=1) - timedelta(days=1)).replace(day=1).strftime('%Y-%m-%d')
SCORE_DATE_LAST_MONTH_END_DASH = ((scoringDate.replace(day=1)) - timedelta(days=1)).strftime('%Y-%m-%d')
SCORE_DATE_LAST_MONTH_YEAR = ((scoringDate.replace(day=1)) - timedelta(days=1)).year
SCORE_DATE_LAST_MONTH_MONTH = ((scoringDate.replace(day=1)) - timedelta(days=1)).month

# validation dates
SCORE_DATE_VAL = valScoringDate.strftime('%Y%m%d')
SCORE_DATE_VAL_DASH = valScoringDate.strftime('%Y-%m-%d')
SCORE_DATE_VAL_MINUS_6_MOS_DASH = ((valScoringDate - relativedelta(months=6)).replace(day=1)).strftime('%Y-%m-%d')
SCORE_DATE_VAL_LAST_MONTH_START_DASH = (valScoringDate.replace(day=1) - timedelta(days=1)).replace(day=1).strftime('%Y-%m-%d')
SCORE_DATE_VAL_LAST_MONTH_END_DASH = ((valScoringDate.replace(day=1)) - timedelta(days=1)).strftime('%Y-%m-%d')
SCORE_DATE_VAL_LAST_MONTH_YEAR = ((valScoringDate.replace(day=1)) - timedelta(days=1)).year
SCORE_DATE_VAL_LAST_MONTH_MONTH = ((valScoringDate.replace(day=1)) - timedelta(days=1)).month

SCORE_DATE_DELTA = 0
SCORE_DATE_VAL_DELTA = 0
TICKET_DATE_WINDOW = 30  # Days of ticket data to be queried

ACCOUNT_CONSL_QUERY_PATH = QUERIES_PATH + 'create_input_account_consl_query.txt'
ACCOUNT_HSIA_DROPS_QUERY_PATH = QUERIES_PATH + 'create_input_account_hsia_drops_query.txt'
ACCOUNT_CALL_DATA_QUERY_PATH = QUERIES_PATH + 'create_input_account_call_data_query.txt'
ACCOUNT_GPON_COPPER_QUERY_PATH = QUERIES_PATH + 'create_input_account_gpon_copper_query.txt'
ACCOUNT_TROUBLE_TICKETS_QUERY_PATH = QUERIES_PATH + 'create_input_account_trouble_tickets_query.txt'
ACCOUNT_PROMO_EXPIRY_QUERY_PATH = QUERIES_PATH + 'create_input_account_promo_expiry_query.txt'
# ACCOUNT_TV_USAGE_QUERY_PATH = QUERIES_PATH + 'create_input_account_tv_usage_query.txt'
ACCOUNT_DEMO_INCOME_QUERY_PATH = QUERIES_PATH + 'create_input_account_demo_income_query.txt'
ACCOUNT_HS_USAGE_QUERY_PATH = QUERIES_PATH + 'create_input_account_hs_usage_query.txt'
ACCOUNT_FFH_BILLING_QUERY_PATH = QUERIES_PATH + 'create_input_account_ffh_billing_query.txt'
ACCOUNT_CLCKSTRM_TELUS_QUERY_PATH = QUERIES_PATH + 'create_input_account_clckstrm_telus_query.txt'
ACCOUNT_TOS_ACTIVE_BANS_QUERY_PATH = QUERIES_PATH + 'create_input_account_tos_active_bans_query.txt'


def create_input_account_consl_view(view_name: str,
                                    score_date: str,
                                    score_date_delta: str,
                                    project_id: str,
                                    dataset_id: str,
                                    region: str,
                                    resource_bucket: str,
                                    query_path: str,
                                    ):

    from google.cloud import bigquery
    from google.cloud import storage

    def if_tbl_exists(client, table_ref):
        from google.cloud.exceptions import NotFound
        try:
            client.get_table(table_ref)
            return True
        except NotFound:
            return False

    def get_gcp_bqclient(project_id, use_local_credential=True):
        token = os.popen('gcloud auth print-access-token').read()
        token = re.sub(f'\n$', '', token)
        credentials = google.oauth2.credentials.Credentials(token)

        bq_client = bigquery.Client(project=project_id)
        if use_local_credential:
            bq_client = bigquery.Client(project=project_id, credentials=credentials)
        return bq_client

    # bq_client = bigquery.Client(project=project_id)
    bq_client = get_gcp_bqclient(project_id)
    dataset = bq_client.dataset(dataset_id)
    table_ref = dataset.table(view_name)

    # load query from .txt file
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(resource_bucket)
    blob = bucket.get_blob(query_path)
    content = blob.download_as_string()
    content = str(content, 'utf-8')

    if if_tbl_exists(bq_client, table_ref):
        bq_client.delete_table(table_ref)

    # content = open(query_path, 'r').read()

    create_base_feature_set_query = content.format(score_date=score_date,
                                                   score_date_delta=score_date_delta,
                                                   view_name=view_name,
                                                   dataset_id=dataset_id,
                                                   project_id=project_id,
                                                   )
    shared_dataset_ref = bq_client.dataset(dataset_id)
    base_feature_set_view_ref = shared_dataset_ref.table(view_name)
    base_feature_set_view = bigquery.Table(base_feature_set_view_ref)
    base_feature_set_view.view_query = create_base_feature_set_query.format(project_id)
    base_feature_set_view = bq_client.create_table(base_feature_set_view)

def create_input_account_ffh_billing_view(view_name: str,
                                              v_report_date: str,
                                              v_start_date: str,
                                              v_end_date: str,
                                              v_bill_year: str,
                                              v_bill_month: str,
                                              dataset_id: str,
                                              project_id: str,
                                              region: str,
                                              resource_bucket: str,
                                              query_path: str
                                              ):
    from google.cloud import bigquery
    from google.cloud import storage

    def get_gcp_bqclient(project_id, use_local_credential=True):
        token = os.popen('gcloud auth print-access-token').read()
        token = re.sub(f'\n$', '', token)
        credentials = google.oauth2.credentials.Credentials(token)

        bq_client = bigquery.Client(project=project_id)
        if use_local_credential:
            bq_client = bigquery.Client(project=project_id, credentials=credentials)
        return bq_client

    # bq_client = bigquery.Client(project=project_id)
    bq_client = get_gcp_bqclient(project_id)
    dataset = bq_client.dataset(dataset_id)
    table_ref = dataset.table(view_name)

    # load query from .txt file
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(resource_bucket)
    blob = bucket.get_blob(query_path)
    content = blob.download_as_string()
    content = str(content, 'utf-8')

    def if_tbl_exists(client, table_ref):
        from google.cloud.exceptions import NotFound
        try:
            client.get_table(table_ref)
            return True
        except NotFound:
            return False

    if if_tbl_exists(bq_client, table_ref):
        bq_client.delete_table(table_ref)

    create_base_feature_set_query = content.format(v_report_date=v_report_date,
                                                   v_start_date=v_start_date,
                                                   v_end_date=v_end_date,
                                                   v_bill_year=v_bill_year,
                                                   v_bill_month=v_bill_month,
                                                   )

    shared_dataset_ref = bq_client.dataset(dataset_id)
    base_feature_set_view_ref = shared_dataset_ref.table(view_name)
    base_feature_set_view = bigquery.Table(base_feature_set_view_ref)
    base_feature_set_view.view_query = create_base_feature_set_query.format(project_id)
    base_feature_set_view = bq_client.create_table(base_feature_set_view)

def create_input_account_hs_usage_view(view_name: str,
                                       v_report_date: str,
                                       v_start_date: str,
                                       v_end_date: str,
                                       v_bill_year: str,
                                       v_bill_month: str,
                                       dataset_id: str,
                                       project_id: str,
                                       region: str,
                                       resource_bucket: str,
                                       query_path: str
                                       ):

    from google.cloud import bigquery
    from google.cloud import storage

    def get_gcp_bqclient(project_id, use_local_credential=True):
        token = os.popen('gcloud auth print-access-token').read()
        token = re.sub(f'\n$', '', token)
        credentials = google.oauth2.credentials.Credentials(token)

        bq_client = bigquery.Client(project=project_id)
        if use_local_credential:
            bq_client = bigquery.Client(project=project_id, credentials=credentials)
        return bq_client

    # bq_client = bigquery.Client(project=project_id)
    bq_client = get_gcp_bqclient(project_id)
    dataset = bq_client.dataset(dataset_id)
    table_ref = dataset.table(view_name)

    # load query from .txt file
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(resource_bucket)
    blob = bucket.get_blob(query_path)
    content = blob.download_as_string()
    content = str(content, 'utf-8')

    def if_tbl_exists(client, table_ref):
        from google.cloud.exceptions import NotFound
        try:
            client.get_table(table_ref)
            return True
        except NotFound:
            return False

    if if_tbl_exists(bq_client, table_ref):
        bq_client.delete_table(table_ref)

    create_base_feature_set_query = content.format(v_report_date=v_report_date,
                                                   v_start_date=v_start_date,
                                                   v_end_date=v_end_date,
                                                   v_bill_year=v_bill_year,
                                                   v_bill_month=v_bill_month,
                                                   )

    shared_dataset_ref = bq_client.dataset(dataset_id)
    base_feature_set_view_ref = shared_dataset_ref.table(view_name)
    base_feature_set_view = bigquery.Table(base_feature_set_view_ref)
    base_feature_set_view.view_query = create_base_feature_set_query.format(project_id)
    base_feature_set_view = bq_client.create_table(base_feature_set_view)

def create_input_account_demo_income_view(view_name: str,
                                          score_date: str,
                                          score_date_delta: str,
                                          dataset_id: str,
                                          project_id: str,
                                          region: str,
                                          resource_bucket: str,
                                          query_path: str
                                          ):

    from google.cloud import bigquery
    from google.cloud import storage

    def get_gcp_bqclient(project_id, use_local_credential=True):
        token = os.popen('gcloud auth print-access-token').read()
        token = re.sub(f'\n$', '', token)
        credentials = google.oauth2.credentials.Credentials(token)

        bq_client = bigquery.Client(project=project_id)
        if use_local_credential:
            bq_client = bigquery.Client(project=project_id, credentials=credentials)
        return bq_client

    # bq_client = bigquery.Client(project=project_id)
    bq_client = get_gcp_bqclient(project_id)
    dataset = bq_client.dataset(dataset_id)
    table_ref = dataset.table(view_name)

    # load query from .txt file
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(resource_bucket)
    blob = bucket.get_blob(query_path)
    content = blob.download_as_string()
    content = str(content, 'utf-8')

    def if_tbl_exists(client, table_ref):
        from google.cloud.exceptions import NotFound
        try:
            client.get_table(table_ref)
            return True
        except NotFound:
            return False

    if if_tbl_exists(bq_client, table_ref):
        bq_client.delete_table(table_ref)

    create_base_feature_set_query = content.format(score_date=score_date,
                                                   score_date_delta=score_date_delta,
                                                   project_id=project_id,
                                                   dataset_id='common_dataset',
                                                   )

    shared_dataset_ref = bq_client.dataset(dataset_id)
    base_feature_set_view_ref = shared_dataset_ref.table(view_name)
    base_feature_set_view = bigquery.Table(base_feature_set_view_ref)
    base_feature_set_view.view_query = create_base_feature_set_query.format(project_id)
    base_feature_set_view = bq_client.create_table(base_feature_set_view)

def create_input_account_promo_expiry_view(view_name: str,
                                           score_date: str,
                                           dataset_id: str,
                                           project_id: str,
                                           region: str,
                                           resource_bucket: str,
                                           query_path: str
                                           ):

    from google.cloud import bigquery
    from google.cloud import storage

    def get_gcp_bqclient(project_id, use_local_credential=True):
        token = os.popen('gcloud auth print-access-token').read()
        token = re.sub(f'\n$', '', token)
        credentials = google.oauth2.credentials.Credentials(token)

        bq_client = bigquery.Client(project=project_id)
        if use_local_credential:
            bq_client = bigquery.Client(project=project_id, credentials=credentials)
        return bq_client

    # bq_client = bigquery.Client(project=project_id)
    bq_client = get_gcp_bqclient(project_id)
    dataset = bq_client.dataset(dataset_id)
    table_ref = dataset.table(view_name)

    # load query from .txt file
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(resource_bucket)
    blob = bucket.get_blob(query_path)
    content = blob.download_as_string()
    content = str(content, 'utf-8')

    def if_tbl_exists(client, table_ref):
        from google.cloud.exceptions import NotFound
        try:
            client.get_table(table_ref)
            return True
        except NotFound:
            return False

    if if_tbl_exists(bq_client, table_ref):
        bq_client.delete_table(table_ref)

    create_base_feature_set_query = content.format(score_date=score_date)

    create_base_feature_set_query = create_base_feature_set_query.replace('{', '{{')
    create_base_feature_set_query = create_base_feature_set_query.replace('}', '}}')

    shared_dataset_ref = bq_client.dataset(dataset_id)
    base_feature_set_view_ref = shared_dataset_ref.table(view_name)
    base_feature_set_view = bigquery.Table(base_feature_set_view_ref)
    base_feature_set_view.view_query = create_base_feature_set_query.format(project_id)
    base_feature_set_view = bq_client.create_table(base_feature_set_view)

def create_input_account_trouble_tickets_view(view_name: str,
                                              score_date: str,
                                              score_date_delta: str,
                                              trouble_tickets_window: str,
                                              dataset_id: str,
                                              project_id: str,
                                              region: str,
                                              resource_bucket: str,
                                              query_path: str
                                              ):

    from google.cloud import bigquery
    from google.cloud import storage

    def get_gcp_bqclient(project_id, use_local_credential=True):
        token = os.popen('gcloud auth print-access-token').read()
        token = re.sub(f'\n$', '', token)
        credentials = google.oauth2.credentials.Credentials(token)

        bq_client = bigquery.Client(project=project_id)
        if use_local_credential:
            bq_client = bigquery.Client(project=project_id, credentials=credentials)
        return bq_client

    # bq_client = bigquery.Client(project=project_id)
    bq_client = get_gcp_bqclient(project_id)
    dataset = bq_client.dataset(dataset_id)
    table_ref = dataset.table(view_name)

    # load query from .txt file
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(resource_bucket)
    blob = bucket.get_blob(query_path)
    content = blob.download_as_string()
    content = str(content, 'utf-8')

    def if_tbl_exists(client, table_ref):
        from google.cloud.exceptions import NotFound
        try:
            client.get_table(table_ref)
            return True
        except NotFound:
            return False

    if if_tbl_exists(bq_client, table_ref):
        bq_client.delete_table(table_ref)

    create_base_feature_set_query = content.format(score_date=score_date,
                                                   score_date_delta=score_date_delta,
                                                   trouble_tickets_window=trouble_tickets_window
                                                   )

    shared_dataset_ref = bq_client.dataset(dataset_id)
    base_feature_set_view_ref = shared_dataset_ref.table(view_name)
    base_feature_set_view = bigquery.Table(base_feature_set_view_ref)
    base_feature_set_view.view_query = create_base_feature_set_query.format(project_id)
    base_feature_set_view = bq_client.create_table(base_feature_set_view)

def create_input_account_gpon_copper_view(view_name: str,
                                          score_date: str,
                                          score_date_delta: str,
                                          dataset_id: str,
                                          project_id: str,
                                          region: str,
                                          resource_bucket: str,
                                          query_path: str
                                          ):

    from google.cloud import bigquery
    from google.cloud import storage

    def get_gcp_bqclient(project_id, use_local_credential=True):
        token = os.popen('gcloud auth print-access-token').read()
        token = re.sub(f'\n$', '', token)
        credentials = google.oauth2.credentials.Credentials(token)

        bq_client = bigquery.Client(project=project_id)
        if use_local_credential:
            bq_client = bigquery.Client(project=project_id, credentials=credentials)
        return bq_client

    # bq_client = bigquery.Client(project=project_id)
    bq_client = get_gcp_bqclient(project_id)
    dataset = bq_client.dataset(dataset_id)
    table_ref = dataset.table(view_name)

    # load query from .txt file
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(resource_bucket)
    blob = bucket.get_blob(query_path)
    content = blob.download_as_string()
    content = str(content, 'utf-8')

    def if_tbl_exists(client, table_ref):
        from google.cloud.exceptions import NotFound
        try:
            client.get_table(table_ref)
            return True
        except NotFound:
            return False

    if if_tbl_exists(bq_client, table_ref):
        bq_client.delete_table(table_ref)

    create_base_feature_set_query = content.format(score_date=score_date,
                                                   score_date_delta=score_date_delta,
                                                   )

    shared_dataset_ref = bq_client.dataset(dataset_id)
    base_feature_set_view_ref = shared_dataset_ref.table(view_name)
    base_feature_set_view = bigquery.Table(base_feature_set_view_ref)
    base_feature_set_view.view_query = create_base_feature_set_query.format(project_id)
    base_feature_set_view = bq_client.create_table(base_feature_set_view)

def create_input_account_call_data_view(view_name: str,
                                        score_date: str,
                                        score_date_delta: str,
                                        dataset_id: str,
                                        project_id: str,
                                        region: str,
                                        resource_bucket: str,
                                        query_path: str
                                        ):

    from google.cloud import bigquery
    from google.cloud import storage

    def get_gcp_bqclient(project_id, use_local_credential=True):
        token = os.popen('gcloud auth print-access-token').read()
        token = re.sub(f'\n$', '', token)
        credentials = google.oauth2.credentials.Credentials(token)

        bq_client = bigquery.Client(project=project_id)
        if use_local_credential:
            bq_client = bigquery.Client(project=project_id, credentials=credentials)
        return bq_client

    # bq_client = bigquery.Client(project=project_id)
    bq_client = get_gcp_bqclient(project_id)
    dataset = bq_client.dataset(dataset_id)
    table_ref = dataset.table(view_name)

    # load query from .txt file
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(resource_bucket)
    blob = bucket.get_blob(query_path)
    content = blob.download_as_string()
    content = str(content, 'utf-8')

    def if_tbl_exists(client, table_ref):
        from google.cloud.exceptions import NotFound
        try:
            client.get_table(table_ref)
            return True
        except NotFound:
            return False

    if if_tbl_exists(bq_client, table_ref):
        bq_client.delete_table(table_ref)

    create_base_feature_set_query = content.format(score_date=score_date,
                                                   score_date_delta=score_date_delta,
                                                   )

    shared_dataset_ref = bq_client.dataset(dataset_id)
    base_feature_set_view_ref = shared_dataset_ref.table(view_name)
    base_feature_set_view = bigquery.Table(base_feature_set_view_ref)
    base_feature_set_view.view_query = create_base_feature_set_query.format(project_id)
    base_feature_set_view = bq_client.create_table(base_feature_set_view)

def create_input_account_hsia_drops_view(view_name: str,
                                         score_date: str,
                                         score_date_delta: str,
                                         dataset_id: str,
                                         project_id: str,
                                         region: str,
                                         resource_bucket: str,
                                         query_path: str
                                         ):

    from google.cloud import bigquery
    from google.cloud import storage

    def get_gcp_bqclient(project_id, use_local_credential=True):
        token = os.popen('gcloud auth print-access-token').read()
        token = re.sub(f'\n$', '', token)
        credentials = google.oauth2.credentials.Credentials(token)

        bq_client = bigquery.Client(project=project_id)
        if use_local_credential:
            bq_client = bigquery.Client(project=project_id, credentials=credentials)
        return bq_client

    # bq_client = bigquery.Client(project=project_id)
    bq_client = get_gcp_bqclient(project_id)
    dataset = bq_client.dataset(dataset_id)
    table_ref = dataset.table(view_name)

    # load query from .txt file
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(resource_bucket)
    blob = bucket.get_blob(query_path)
    content = blob.download_as_string()
    content = str(content, 'utf-8')

    def if_tbl_exists(client, table_ref):
        from google.cloud.exceptions import NotFound
        try:
            client.get_table(table_ref)
            return True
        except NotFound:
            return False

    if if_tbl_exists(bq_client, table_ref):
        bq_client.delete_table(table_ref)

    create_base_feature_set_query = content.format(score_date=score_date,
                                                   score_date_delta=score_date_delta,
                                                   project_id=project_id,
                                                   dataset_id='common_dataset',
                                                   )

    shared_dataset_ref = bq_client.dataset(dataset_id)
    base_feature_set_view_ref = shared_dataset_ref.table(view_name)
    base_feature_set_view = bigquery.Table(base_feature_set_view_ref)
    base_feature_set_view.view_query = create_base_feature_set_query.format(project_id)
    base_feature_set_view = bq_client.create_table(base_feature_set_view)
    


def create_input_account_clckstrm_telus_view(view_name: str,
                                    score_date: str,
                                    score_date_delta: str,
                                    project_id: str,
                                    dataset_id: str,
                                    region: str,
                                    resource_bucket: str,
                                    query_path: str,
                                    ):

    from google.cloud import bigquery
    from google.cloud import storage

    def if_tbl_exists(client, table_ref):
        from google.cloud.exceptions import NotFound
        try:
            client.get_table(table_ref)
            return True
        except NotFound:
            return False

    def get_gcp_bqclient(project_id, use_local_credential=True):
        token = os.popen('gcloud auth print-access-token').read()
        token = re.sub(f'\n$', '', token)
        credentials = google.oauth2.credentials.Credentials(token)

        bq_client = bigquery.Client(project=project_id)
        if use_local_credential:
            bq_client = bigquery.Client(project=project_id, credentials=credentials)
        return bq_client

    # bq_client = bigquery.Client(project=project_id)
    bq_client = get_gcp_bqclient(project_id)
    dataset = bq_client.dataset(dataset_id)
    table_ref = dataset.table(view_name)

    # load query from .txt file
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(resource_bucket)
    blob = bucket.get_blob(query_path)
    content = blob.download_as_string()
    content = str(content, 'utf-8')

    if if_tbl_exists(bq_client, table_ref):
        bq_client.delete_table(table_ref)

    # content = open(query_path, 'r').read()

    create_base_feature_set_query = content.format(score_date=score_date,
                                                   score_date_delta=score_date_delta,
                                                   view_name=view_name,
                                                   dataset_id=dataset_id,
                                                   project_id=project_id,
                                                   )
    shared_dataset_ref = bq_client.dataset(dataset_id)
    base_feature_set_view_ref = shared_dataset_ref.table(view_name)
    base_feature_set_view = bigquery.Table(base_feature_set_view_ref)
    base_feature_set_view.view_query = create_base_feature_set_query.format(project_id)
    base_feature_set_view = bq_client.create_table(base_feature_set_view)
    
def create_input_account_tos_active_bans_view(view_name: str,
                                    score_date: str,
                                    score_date_delta: str,
                                    v_start_date: str,
                                    v_end_date: str,
                                    project_id: str,
                                    dataset_id: str,
                                    region: str,
                                    resource_bucket: str,
                                    query_path: str,
                                    ):

    from google.cloud import bigquery
    from google.cloud import storage

    def if_tbl_exists(client, table_ref):
        from google.cloud.exceptions import NotFound
        try:
            client.get_table(table_ref)
            return True
        except NotFound:
            return False


    def get_gcp_bqclient(project_id, use_local_credential=True):
        token = os.popen('gcloud auth print-access-token').read()
        token = re.sub(f'\n$', '', token)
        credentials = google.oauth2.credentials.Credentials(token)

        bq_client = bigquery.Client(project=project_id)
        if use_local_credential:
            bq_client = bigquery.Client(project=project_id, credentials=credentials)
        return bq_client

    # bq_client = bigquery.Client(project=project_id)
    bq_client = get_gcp_bqclient(project_id)
    dataset = bq_client.dataset(dataset_id)
    table_ref = dataset.table(view_name)

    # load query from .txt file
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(resource_bucket)
    blob = bucket.get_blob(query_path)
    content = blob.download_as_string()
    content = str(content, 'utf-8')

    if if_tbl_exists(bq_client, table_ref):
        bq_client.delete_table(table_ref)

    # content = open(query_path, 'r').read()

    create_base_feature_set_query = content.format(score_date=score_date,
                                                   score_date_delta=score_date_delta,
                                                   v_start_date=v_start_date,
                                                   v_end_date=v_end_date,
                                                   view_name=view_name,
                                                   dataset_id=dataset_id,
                                                   project_id=project_id,
                                                   )
    shared_dataset_ref = bq_client.dataset(dataset_id)
    base_feature_set_view_ref = shared_dataset_ref.table(view_name)
    base_feature_set_view = bigquery.Table(base_feature_set_view_ref)
    base_feature_set_view.view_query = create_base_feature_set_query.format(project_id)
    base_feature_set_view = bq_client.create_table(base_feature_set_view)


#---------------------------------------------------------------------------------------------------------------------------

    
def preprocess(
        account_consl_view: str,
        account_bill_view: str,
        hs_usage_view: str,
        demo_income_view: str,
        promo_expiry_view: str,
        trouble_tickets_view: str,
        gpon_copper_view: str,
        call_data_view: str,
        hsia_drops_view: str,
        clckstrm_telus_view: str, 
        tos_active_bans_view: str, 
        save_data_path: str,
        project_id: str,
        dataset_id: str
):

    from google.cloud import bigquery
    import pandas as pd
    import gc
    import time
    
    def get_gcp_bqclient(project_id, use_local_credential=True):
        token = os.popen('gcloud auth print-access-token').read()
        token = re.sub(f'\n$', '', token)
        credentials = google.oauth2.credentials.Credentials(token)

        bq_client = bigquery.Client(project=project_id)
        if use_local_credential:
            bq_client = bigquery.Client(project=project_id, credentials=credentials)
        return bq_client

    client = get_gcp_bqclient(project_id)

    consl_data_set = f"{project_id}.{dataset_id}.{account_consl_view}" #`divg-churn-analysis-pr-7e40f6.divg_churn_analysis_pr_dataset.internet_churn_pipeline_consl_data_training_bi_layer`

    build_df_consl = '''SELECT * FROM `{consl_data_set}`'''.format(consl_data_set=consl_data_set)
    df_consl = client.query(build_df_consl).to_dataframe() #df_consl is the dataframe that contains consolidated input data (dataset #1)
    print('......base data done')

    # product mix
    df_mix = df_consl[[
        'ban',
        'product_mix_all',
        'sing_count',
        'hsic_count',
        'mob_count',
        'shs_count',
        'ttv_count',
        'stv_count',
        'diic_count',
        'new_c_ind',
        'new_sing_ind',
        'new_hsic_ind',
        'new_ttv_ind',
        'new_smhm_ind',
        'mnh_ind',
    ]]
    df_mix = df_mix.drop_duplicates(subset=['ban']).set_index('ban').add_prefix('productMix_')

    # df_join
    df_join = df_mix.fillna(0)

    del df_mix
    gc.collect()
    print('......product mix done')

    bill_data_set = f"{project_id}.{dataset_id}.{account_bill_view}" #`divg-churn-analysis-pr-7e40f6.divg_churn_analysis_pr_dataset.internet_churn_pipeline_ffh_billing_data_training_bi_layer`
    build_df_bill = '''SELECT * FROM `{bill_data_set}`'''.format(bill_data_set=bill_data_set)

    client = get_gcp_bqclient(project_id)
    df_bill = client.query(build_df_bill).to_dataframe() #df_bill is the dataframe that contains billing input data (dataset #2)

    df_bill = df_bill.set_index('ban').add_prefix('ffhBill_')

    # df_join
    df_join = df_join.join(df_bill).fillna(0) #join datasets #1 and #2 = dataset 1+2
    del df_bill
    gc.collect()
    print('......account bill done')

    hs_usage_data_set = f"{project_id}.{dataset_id}.{hs_usage_view}" #`divg-churn-analysis-pr-7e40f6.divg_churn_analysis_pr_dataset.internet_churn_pipeline_hs_usage_data_training_bi_layer`
    build_df_hs_usage = '''SELECT * FROM `{hs_usage_data_set}`'''.format(hs_usage_data_set=hs_usage_data_set)

    client = get_gcp_bqclient(project_id)
    df_hs_usage = client.query(build_df_hs_usage).to_dataframe() #df_hs_usage is the dataframe that contains internet usage input data (dataset #3)

    df_hs_usage = df_hs_usage.set_index('ban').add_prefix('hsiaUsage_')

    # df_join
    df_join = df_join.join(df_hs_usage).fillna(0) #join datasets#1+2 and #3
    del df_hs_usage
    gc.collect()
    print('......hs usage done')

    demo_income_data_set = f"{project_id}.{dataset_id}.{demo_income_view}" #`divg-churn-analysis-pr-7e40f6.divg_churn_analysis_pr_dataset.internet_churn_pipeline_demo_income_data_training_bi_layer`
    build_df_demo_income = '''SELECT * FROM `{demo_income_data_set}`'''.format(
        demo_income_data_set=demo_income_data_set)
    
    client = get_gcp_bqclient(project_id)
    df_income = client.query(build_df_demo_income).to_dataframe()

    df_income = df_income.set_index('ban')
    df_income['demo_urban_flag'] = df_income.demo_sgname.str.lower().str.contains('urban').fillna(0).astype(int)
    df_income['demo_rural_flag'] = df_income.demo_sgname.str.lower().str.contains('rural').fillna(0).astype(int)
    df_income['demo_family_flag'] = df_income.demo_lsname.str.lower().str.contains('families').fillna(0).astype(int)
    df_income_dummies = pd.get_dummies(df_income[['demo_lsname']])
    df_income_dummies.columns = df_income_dummies.columns.str.replace('&', 'and')
    df_income_dummies.columns = df_income_dummies.columns.str.replace(' ', '_')
    df_income = df_income[['demo_avg_income', 'demo_urban_flag', 'demo_rural_flag', 'demo_family_flag']].join(
        df_income_dummies)
    df_income.demo_avg_income = df_income.demo_avg_income.astype(float)
    df_income.demo_avg_income = df_income.demo_avg_income.fillna(df_income.demo_avg_income.median())
    df_group_income = df_income.groupby('ban').agg('mean')
    df_group_income = df_group_income.add_prefix('demographics_')
    # df_join
    df_join = df_join.join(df_group_income.fillna(df_group_income.median()))

    del df_group_income
    del df_income
    gc.collect()
    print('......income done')

    promo_expiry_data_set = f"{project_id}.{dataset_id}.{promo_expiry_view}" #`divg-churn-analysis-pr-7e40f6.divg_churn_analysis_pr_dataset.internet_churn_pipeline_promo_expiry_data_training_bi_layer`
    build_df_promo = '''SELECT * FROM `{promo_expiry_data_set}` '''.format(
        promo_expiry_data_set=promo_expiry_data_set)
    
    client = get_gcp_bqclient(project_id)
    df_promo = client.query(build_df_promo).to_dataframe()  # Make an API request.

    df_promo = df_promo.set_index('ban')
    disc_cols = [col for col in df_promo.columns if 'disc' in col]
    bill_cols = [col for col in df_promo.columns if 'disc' not in col]

    df_join = df_join.join(df_promo[disc_cols].add_prefix('promo_'))
    df_join = df_join.join(df_promo[bill_cols].add_prefix('ffhBill_')).fillna(0)

    del df_promo
    gc.collect()
    print('......promo expiry done')

#     trouble_tickets_data_set = f"{project_id}.{dataset_id}.{trouble_tickets_view}" #`_churn_pipeline_trouble_tickets_data_training_bi_layer`
#     build_df_trouble_tickets = '''
#     SELECT * FROM `{trouble_tickets_data_set}`
#     '''.format(trouble_tickets_data_set=trouble_tickets_data_set)
    
#     client = get_gcp_bqclient(project_id)
#     df_trouble_tickets = client.query(build_df_trouble_tickets).to_dataframe()

#     df_trouble_tickets = df_trouble_tickets.set_index('ban')
#     df_join = df_join.join(df_trouble_tickets.add_prefix('troubleTickets_')).fillna(0)
#     del df_trouble_tickets
#     gc.collect()
#     print('......trouble tickets done')
    
    # gpon copper
    gpon_copper_data_set = f"{project_id}.{dataset_id}.{gpon_copper_view}"
    build_df_gpon_copper = '''
    SELECT * FROM `{gpon_copper_data_set}` 
    '''.format(gpon_copper_data_set=gpon_copper_data_set)
    
    client = get_gcp_bqclient(project_id)
    df_gpon_copper = client.query(build_df_gpon_copper).to_dataframe()

    df_gpon_copper = df_gpon_copper.set_index('ban')
    df_join = df_join.join(df_gpon_copper.add_prefix('infra_')).fillna(0)
    del df_gpon_copper
    gc.collect()
    print('......gpon copper done')
    
#     # call data
#     call_data_set = f"{project_id}.{dataset_id}.{call_data_view}"
#     build_df_call_data = '''SELECT * FROM `{call_data_set}`'''.format(call_data_set=call_data_set)
    
#     client = get_gcp_bqclient(project_id)
#     df_call_data = client.query(build_df_call_data).to_dataframe()  # Make an API request.

    # # df_join
    # df_call_data = df_call_data.set_index('ban')
    # df_join = df_join.join(df_call_data.add_prefix('callCentre_')).fillna(0)
    # del df_call_data
    # gc.collect()
    # print('......call data done')
    
#     # hsia drops
#     hsia_drops_data_set = f"{project_id}.{dataset_id}.{hsia_drops_view}" #`_churn_pipeline_hsia_drops_training_bi_layer`
#     build_df_hsia_drops = '''
#     SELECT * FROM `{hsia_drops_data_set}` 
#     '''.format(hsia_drops_data_set=hsia_drops_data_set)
    
#     client = get_gcp_bqclient(project_id)
#     df_hsia_drops = client.query(build_df_hsia_drops).to_dataframe()

#     df_hsia_drops = df_hsia_drops.set_index('ban')
#     df_join = df_join.join(df_hsia_drops.add_prefix('hsiaDrops_')).fillna(0)
#     del df_hsia_drops
#     gc.collect()
#     print('......hsia drop done')

    # clickstream data
    clckstrm_telus_data_set = f"{project_id}.{dataset_id}.{clckstrm_telus_view}" 
    build_df_clckstrm_telus = '''SELECT * FROM `{clckstrm_telus_data_set}`'''.format(clckstrm_telus_data_set=clckstrm_telus_data_set)

    client = get_gcp_bqclient(project_id)
    df_clckstrm_telus = client.query(build_df_clckstrm_telus).to_dataframe() 

    df_clckstrm_telus = df_clckstrm_telus.set_index('ban').add_prefix('clckstrmData_')

    # df_join
    df_join = df_join.join(df_clckstrm_telus).fillna(0) 
    del df_clckstrm_telus
    gc.collect()
    print('......clcktsrm data done')
    
    # tos active bans
    tos_active_bans = f"{project_id}.{dataset_id}.{tos_active_bans_view}" 
    build_df_tos_active_data = '''SELECT * FROM `{tos_active_bans}`'''.format(tos_active_bans=tos_active_bans)

    client = get_gcp_bqclient(project_id)
    df_tos_active_data = client.query(build_df_tos_active_data).to_dataframe()  # Make an API request.

    # df_join
    df_tos_active_data = df_tos_active_data.set_index('ban')
    df_join = df_join.join(df_tos_active_data).fillna(0)
    del df_tos_active_data
    gc.collect()
    print('......tos active data done')
    
    df_join.columns = df_join.columns.str.replace(' ', '_')
    df_join.columns = df_join.columns.str.replace('-', '_')

    df_final = df_join.copy()
    del df_join
    gc.collect()
    print('......df final done')

    for f in df_final.columns:
        df_final[f] = list(df_final[f])

    # df_final = df_final.loc[(df_final['tos_ind'] == 0) & (df_final['productMix_hsic_count'] >= 1)].reset_index()
    df_final = df_final.loc[(df_final['tos_ind'] == 0) & (df_final['productMix_new_smhm_ind'] == 0)].reset_index()
    df_final = df_final.drop(['tos_ind', 'productMix_new_smhm_ind'], axis=1) 
    
    df_final.to_csv(save_data_path, index=False, compression='gzip') 
    del df_final
    gc.collect()
    print(f'......csv saved in {save_data_path}')
    time.sleep(300)

#---------------------------------------------------------------------------------------------------------------------------
    
def pipeline(
            project_id: str = PROJECT_ID,
            region: str = REGION,
            resource_bucket: str = RESOURCE_BUCKET,
            file_bucket: str = FILE_BUCKET
    ):

        # --------------- validation view ops ---------------
        create_input_account_consl_validation_view_op = create_input_account_consl_view(
            view_name=CONSL_VIEW_VALIDATION_NAME, 
            score_date=SCORE_DATE_VAL,
            score_date_delta=SCORE_DATE_VAL_DELTA,
            project_id=PROJECT_ID,
            dataset_id=DATASET_ID,
            region=REGION,
            resource_bucket=RESOURCE_BUCKET,
            query_path=ACCOUNT_CONSL_QUERY_PATH,
        )

        create_input_account_ffh_billing_validation_view_op = create_input_account_ffh_billing_view(
            v_report_date=SCORE_DATE_VAL_DASH,
            v_start_date=SCORE_DATE_VAL_MINUS_6_MOS_DASH,
            v_end_date=SCORE_DATE_VAL_LAST_MONTH_END_DASH,
            v_bill_year=SCORE_DATE_VAL_LAST_MONTH_YEAR,
            v_bill_month=SCORE_DATE_VAL_LAST_MONTH_MONTH,
            view_name=FFH_BILLING_VIEW_VALIDATION_NAME,
            dataset_id=DATASET_ID,
            project_id=PROJECT_ID,
            region=REGION,
            resource_bucket=RESOURCE_BUCKET,
            query_path=ACCOUNT_FFH_BILLING_QUERY_PATH
        )

        create_input_account_hs_usage_validation_view_op = create_input_account_hs_usage_view(
            v_report_date=SCORE_DATE_VAL_DASH,
            v_start_date=SCORE_DATE_VAL_MINUS_6_MOS_DASH,
            v_end_date=SCORE_DATE_VAL_LAST_MONTH_END_DASH,
            v_bill_year=SCORE_DATE_VAL_LAST_MONTH_YEAR,
            v_bill_month=SCORE_DATE_VAL_LAST_MONTH_MONTH,
            view_name=HS_USAGE_VIEW_VALIDATION_NAME,
            dataset_id=DATASET_ID,
            project_id=PROJECT_ID,
            region=REGION,
            resource_bucket=RESOURCE_BUCKET,
            query_path=ACCOUNT_HS_USAGE_QUERY_PATH
        )

        create_input_account_demo_income_validation_view_op = create_input_account_demo_income_view(
            score_date=SCORE_DATE_VAL,
            score_date_delta=SCORE_DATE_VAL_DELTA,
            view_name=DEMO_INCOME_VIEW_VALIDATION_NAME,
            dataset_id=DATASET_ID,
            project_id=PROJECT_ID,
            region=REGION,
            resource_bucket=RESOURCE_BUCKET,
            query_path=ACCOUNT_DEMO_INCOME_QUERY_PATH
        )

        create_input_account_promo_expiry_validation_view_op = create_input_account_promo_expiry_view(
            score_date=SCORE_DATE_VAL,
            view_name=PROMO_EXPIRY_VIEW_VALIDATION_NAME,
            dataset_id=DATASET_ID,
            project_id=PROJECT_ID,
            region=REGION,
            resource_bucket=RESOURCE_BUCKET,
            query_path=ACCOUNT_PROMO_EXPIRY_QUERY_PATH
        )

        create_input_account_trouble_tickets_validation_view_op = create_input_account_trouble_tickets_view(
            score_date=SCORE_DATE_VAL,
            score_date_delta=SCORE_DATE_VAL_DELTA,
            trouble_tickets_window=TICKET_DATE_WINDOW,
            view_name=TROUBLE_TICKETS_VIEW_VALIDATION_NAME,
            dataset_id=DATASET_ID,
            project_id=PROJECT_ID,
            region=REGION,
            resource_bucket=RESOURCE_BUCKET,
            query_path=ACCOUNT_TROUBLE_TICKETS_QUERY_PATH
        )

        create_input_account_gpon_copper_validation_view_op = create_input_account_gpon_copper_view(
            score_date=SCORE_DATE_VAL,
            score_date_delta=SCORE_DATE_VAL_DELTA,
            view_name=GPON_COPPER_VIEW_VALIDATION_NAME,
            dataset_id=DATASET_ID,
            project_id=PROJECT_ID,
            region=REGION,
            resource_bucket=RESOURCE_BUCKET,
            query_path=ACCOUNT_GPON_COPPER_QUERY_PATH
        )

        create_input_account_call_data_validation_view_op = create_input_account_call_data_view(
            score_date=SCORE_DATE_VAL,
            score_date_delta=SCORE_DATE_VAL_DELTA,
            view_name=CALL_DATA_VIEW_VALIDATION_NAME,
            dataset_id=DATASET_ID,
            project_id=PROJECT_ID,
            region=REGION,
            resource_bucket=RESOURCE_BUCKET,
            query_path=ACCOUNT_CALL_DATA_QUERY_PATH
        )

        create_input_account_hsia_drops_validation_view_op = create_input_account_hsia_drops_view(
            score_date=SCORE_DATE_VAL,
            score_date_delta=SCORE_DATE_VAL_DELTA,
            view_name=HSIA_DROPS_VIEW_VALIDATION_NAME,
            dataset_id=DATASET_ID,
            project_id=PROJECT_ID,
            region=REGION,
            resource_bucket=RESOURCE_BUCKET,
            query_path=ACCOUNT_HSIA_DROPS_QUERY_PATH
        )
        
        create_input_account_clckstrm_telus_validation_view_op = create_input_account_clckstrm_telus_view(
            score_date=SCORE_DATE_VAL,
            score_date_delta=SCORE_DATE_VAL_DELTA,
            view_name=CLCKSTRM_TELUS_VIEW_VALIDATION_NAME,
            dataset_id=DATASET_ID,
            project_id=PROJECT_ID,
            region=REGION,
            resource_bucket=RESOURCE_BUCKET,
            query_path=ACCOUNT_CLCKSTRM_TELUS_QUERY_PATH
        )
        
        create_input_account_tos_active_bans_validation_view_op = create_input_account_tos_active_bans_view(
            score_date=SCORE_DATE,
            score_date_delta=SCORE_DATE_DELTA,
            v_start_date=SCORE_DATE_LAST_MONTH_START_DASH,
            v_end_date=SCORE_DATE_LAST_MONTH_END_DASH,
            view_name=TOS_ACTIVE_BANS_VIEW_NAME,
            dataset_id=DATASET_ID,
            project_id=PROJECT_ID,
            region=REGION,
            resource_bucket=RESOURCE_BUCKET,
            query_path=ACCOUNT_TOS_ACTIVE_BANS_QUERY_PATH
        )

        # ----- preprocessing train data --------
        preprocess_validation_op = preprocess(
            account_consl_view=CONSL_VIEW_VALIDATION_NAME,
            account_bill_view=FFH_BILLING_VIEW_VALIDATION_NAME,
            hs_usage_view=HS_USAGE_VIEW_VALIDATION_NAME,
            demo_income_view=DEMO_INCOME_VIEW_VALIDATION_NAME,
            promo_expiry_view=PROMO_EXPIRY_VIEW_VALIDATION_NAME,
            trouble_tickets_view=TROUBLE_TICKETS_VIEW_VALIDATION_NAME,
            gpon_copper_view=GPON_COPPER_VIEW_VALIDATION_NAME,
            call_data_view=CALL_DATA_VIEW_VALIDATION_NAME,
            hsia_drops_view=HSIA_DROPS_VIEW_VALIDATION_NAME,
            clckstrm_telus_view=CLCKSTRM_TELUS_VIEW_VALIDATION_NAME, 
            tos_active_bans_view=TOS_ACTIVE_BANS_VIEW_NAME, 
            save_data_path='gs://{}/{}/{}_validation.csv.gz'.format(FILE_BUCKET, SERVICE_TYPE, SERVICE_TYPE),
            project_id=PROJECT_ID,
            dataset_id=DATASET_ID,
        )
        
        create_input_account_consl_validation_view_op
        create_input_account_ffh_billing_validation_view_op
        create_input_account_hs_usage_validation_view_op
        create_input_account_demo_income_validation_view_op
        create_input_account_promo_expiry_validation_view_op
        create_input_account_trouble_tickets_validation_view_op
        create_input_account_gpon_copper_validation_view_op
        create_input_account_call_data_validation_view_op
        create_input_account_hsia_drops_validation_view_op
        create_input_account_clckstrm_telus_validation_view_op
        create_input_account_tos_active_bans_validation_view_op

        preprocess_validation_op
        
        return pipeline

In [ ]:
pipeline(project_id = PROJECT_ID, region = REGION, resource_bucket = RESOURCE_BUCKET, file_bucket = FILE_BUCKET)